In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
#from functions import get_zone, load_360_data, get_progressive_actions, zone_polygon, plot_team_zone_shares, calc_width, calc_directness, calc_tempo, calc_possession_time
import functions
import plotting_func
import seaborn as sns
import pandas as pd

pd.set_option("display.float_format", "{:,.4f}".format)


### Features to use

Main tactical features to use:
- Width offensivly ✅
Average distance from center-line (goal-to-goal) where progressive actions end. 
- Directness ✅
Relationship of progressive actions to non progressive actions
- Tempo ✅
Velocity of ball, how fast do actions happen?
- Press height✅
How high up the field does the average pressure happen?
- Press intensity✅
How many pressing actions per minute of opposition possession
- Pass length✅
How long is the average pass, and how much does it deviate?

- Restförsvar ?
- Fasta situationer ?

- Höjd på backlinje
- Antal progressiva passningar

- Bollhastighet
- Final third entries

### Game State:
- Which zone are we in?
- Distance to goal?
- Angle to goal?
- Opponents near?
- How many opponants near?
- Score?
-  Red cards?

### Tactics: 


In [2]:
actions=("Pass", "Carry", "Dribble")
action_types_exclude = ["Throw-in", "Corner", "Free Kick", "Kick-off", "Goal Kick"]
cool = functions.load_360_data(actions, three_sixty_only=True)

print("data loaded")

#is_dead_ball_action = df["pass_type"].isin(action_types_exclude)


actions = functions.get_progressive_actions(df=cool)
prog_passes = actions[actions["is_progressive"] == False]



Loading events: 100%|██████████| 326/326 [00:15<00:00, 21.66it/s]


data loaded


In [8]:
df = cool.copy()

In [4]:
poss = functions.calc_possession_time(df, match=True)
print(poss)
print("total minutes:", poss["possession_seconds"].sum()/60)



     match_id  team_id  possession_seconds
0     3788741      909          1,619.6264
1     3788741      914          3,039.8754
2     3788742      776          3,166.3820
3     3788742     1835          1,413.5935
4     3788743      782          2,934.3190
..        ...      ...                 ...
647   4020005      865          3,546.7729
648   4020077      857          2,452.6899
649   4020077      863          3,808.8964
650   4020846      863          3,920.0052
651   4020846      865          2,438.0378

[652 rows x 3 columns]
total minutes: 25940.89560715916


In [5]:
seg = functions.calc_possession_time(df, match=True)  # if you already computed seg inside, use that

# Better: replicate your internal seg table quickly:
df = df.copy()
ts = pd.to_timedelta(df["timestamp"].astype(str), errors="coerce").dt.total_seconds().fillna(0.0)
t_end = ts + pd.to_numeric(df.get("duration", 0), errors="coerce").fillna(0.0).to_numpy()

per_len = (pd.DataFrame({"match_id": df["match_id"], "period": df["period"], "t_end": t_end})
           .groupby(["match_id","period"], as_index=False)["t_end"].max()
           .sort_values(["match_id","period"]))
per_len["offset_sec"] = per_len.groupby("match_id")["t_end"].shift(1).fillna(0.0)
per_len["offset_sec"] = per_len.groupby("match_id")["offset_sec"].cumsum()
df = df.merge(per_len[["match_id","period","offset_sec"]], on=["match_id","period"], how="left")

df["t_abs"] = ts + df["offset_sec"].fillna(0.0)
df["t_end_abs"] = t_end + df["offset_sec"].fillna(0.0)

seg = (df.groupby(["match_id","period","possession","possession_team_id"], as_index=False)
         .agg(start=("t_abs","min"), end=("t_end_abs","max")))
seg["secs"] = (seg["end"] - seg["start"]).clip(lower=0)

seg.sort_values("secs", ascending=False).head(20)



,match_id,period,possession,possession_team_id,start,end,secs
26915,3869685,2,196,771,"3,154.8440","6,372.4900","3,217.6460"
9165,3795506,2,158,768,"2,949.8597","6,036.1940","3,086.3343"
6680,3794686,2,190,785,"2,769.2850","5,829.3100","3,060.0250"
21932,3857291,1,71,772,0.5860,"3,005.1440","3,004.5580"
9088,3795506,1,82,914,1.6490,"2,949.5497","2,947.9007"
450,3788743,1,90,782,0.1590,"2,942.6590","2,942.5000"
19998,3857280,1,72,781,0.8680,"2,939.1685","2,938.3005"
8459,3795187,1,56,768,0.6620,"2,907.7920","2,907.1300"
13730,3835342,1,110,861,0.6340,"2,885.8820","2,885.2480"
7472,3794691,1,72,773,0.5700,"2,885.4630","2,884.8930"


In [12]:
m,p,poss,tid = 3857291, 1, 71, 772
sub = df[(df.match_id==m) & (df.period==p) &
                (df.possession==poss) & (df.possession_team_id==tid)].copy()

print("rows:", len(sub))
print("t min/max:", sub["t_abs_sec"].min(), sub["t_abs_sec"].max())
print("unique possession in sub:", sub["possession"].nunique())
print("unique teams in sub:", sub["possession_team_id"].nunique())
sub[["timestamp","minute","second","type","possession","possession_team_id"]].head(10)
sub[["timestamp","minute","second","type","possession","possession_team_id"]].tail(10)


rows: 12


KeyError: 't_abs_sec'

In [16]:
mids = df["match_id"].dropna().astype(int).unique()

totals = []
for m in mids[:50]:  # take first 50 to start
    o = functions.calc_possession_time(df[df.match_id==m], match=True, max_gap_s=10.0)
    totals.append(o["possession_seconds"].sum()/60)

s = pd.Series(totals)
print(s.describe())


count   50.0000
mean    57.3660
std      7.8491
min     42.5956
25%     51.8293
50%     55.8612
75%     60.2019
max     77.0926
dtype: float64
